# 对文献进行提取并且分块

In [1]:
import fitz
import glob
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings,HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate

读取文献

In [2]:
result = glob.glob('./pdf_sources/*.pdf')

读取pdf为字符文本

In [6]:
pdf_pageText = []
meta_info = []
# 关键词列表
pdf_keywords = []
# 摘要列表
pdf_subject = []
allpageText = ""

for pdf in result :
    try :
        ipageText = ""
        pdfdoc = fitz.open(pdf)
        for page in pdfdoc:
            ipageText += page.get_text("text")
        allpageText += ipageText
        pdf_pageText.append(ipageText)
        # 生成元数据
        ipm = pdfdoc.metadata
        # 生成关键词和摘要列表
        pdf_keywords.append(ipm["keywords"])
        pdf_subject.append(ipm["subject"])
    except :
        print("出错了!...")

出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...
出错了!...


对字符文本进行分块处理

In [7]:
# text_splitter = CharacterTextSplitter(
#     separator = "\n",
#     chunk_size = 1000,
#     chunk_overlap = 200,
#     length_function=len
# )
# chunks = text_splitter.split_text(allpageText)

chunks = pdf_keywords + pdf_subject

user提出一个quesiton，首先与prompt进行相似度匹配，

数据持久化

In [8]:
import pandas as pd
# 将处理好的分块保留为pageText.json格式的文件
pd.DataFrame(chunks).to_json('pageText.json')

json_path = "F:\\工作以及比赛\\大一统框架\\dd\\try2\\app\\backproject\\app01\\chatapi\\pageText.json"
# 读取pageText.json格式的文件
chunks = pd.read_json(json_path).iloc[:,0].tolist()

# 进行向量相似度分析，给出文本分块

In [393]:
from InstructorEmbedding import INSTRUCTOR
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 读取本地模型文件
model = INSTRUCTOR("F:\\工作以及比赛\\大一统框架\\dd\\try2\\t1\\model")

load INSTRUCTOR_Transformer
max_seq_length  512


对分块进行编码处理

In [10]:
s1 = model.encode(chunks)

编码持久化

In [11]:
# 将encode保存为chunksEncoding.json文件
pd.DataFrame(s1).to_json("./chunksEncoding.json")

# 读取chunksEncoding.json文件
s1 = np.array(pd.read_json("./chunksEncoding.json"))

给定问题

In [12]:
question = "What are the core microorganisms in anaerobic digestion?"

编码问题

In [13]:
s2 = model.encode(question)

In [17]:
# s1.shape

s2.shape

(768,)

进行向量相似化检索

In [8]:
max_len = 10
document = np.array(chunks)[cosine_similarity(s1,s2.reshape(1,-1)).squeeze().argsort()[0:max_len].tolist()].tolist()

# 问题匹配

In [99]:
prompt_title = np.array(["Anaerobic digestion (material addition effects)",
                    "Microbial effects of anaerobic digestion",
                    "Analysis of the nature of anaerobic digestion systems",
                    "Impact of anaerobic digestion on gas production",])

## 基于两层相似度检索

In [101]:
def question_match(question) :
    # 对问题进行编码处理
    question_encode = model.encode(question)
    # 第一层检索
    # 根据每个prompt的title进行优先匹配
    prompt_title = np.array(["Anaerobic digestion (material addition effects)",
                        "Microbial effects of anaerobic digestion",
                        "Analysis of the nature of anaerobic digestion systems",
                        "Impact of anaerobic digestion on gas production",])
    title_encode = model.encode(prompt_title)
    one_vector = cosine_similarity(title_encode,question_encode.reshape(1,-1))
    # 第二层检索
    # 根据每个prompt的content进行二次匹配
        # 给出文件列表
    dir_list = [f'./t{i}.txt' for i in range(1,5)]
    content_list = []
        # 读取四种类型的prompt
    for dir in dir_list :
        with open(dir , 'r' , encoding='utf8') as f :
            content = f.read()
        content_list.append(content)
    content_list = model.encode(np.array(content_list))
    two_vector = cosine_similarity(content_list,question_encode.reshape(1,-1))

    # print(f"第一层认为第{one_vector.argmax()+1}个最匹配,第二层认为第{two_vector.argmax()+1}个最匹配,综合认为第{(0.8 * one_vector + 0.2 * two_vector).argmax() + 1}个最匹配")
    return (0.8 * one_vector + 0.2 * two_vector).argmax() + 1

In [102]:
# question_list = [
#     "Direct electron transfer promotes anaerobic digestion mechanism?",
#     "How to further improve the efficiency of anaerobic digestion, such as processing the straw load to the rumen.",
#     "What are the core microorganisms in anaerobic digestion?",
#     "Does oxygen improve anaerobic digestion efficiency?",
#     "Is around 40 degrees suitable for efficient anaerobic methane production?",
#     "Effect of Methanocelleus on anaerobic digestion of food waste? ",
#     "What effect does the addition of biochar have on the biogas production of anaerobic digestion of food waste? What should its optimal addition amount be?"
# ]

In [130]:
import numpy as np

# 读取quesiton_list
question_dir_list = [f'./question_list/question_list{i}.txt' for i in range(1,5)]
question_list_gpt = []

for iquestion_list in question_dir_list :
    with open(iquestion_list,'r') as f :
        question_list_content = f.read()
    question_list_gpt += question_list_content.split('\n')

# 对问题进行去重处理
question_list = list(set(question_list_gpt))

# 给定一个乱序问题列表
question_list_copy = list(question_list)

# 打乱顺序
np.random.shuffle(question_list_copy)

In [147]:
show_list = []
for iquestion in question_list_copy :
    question_index = question_match(iquestion)
    source_index = question_list.index(iquestion) // 100 + 1
    show_list.append([iquestion,question_index,source_index,prompt_title[question_index-1]])

In [148]:
pd.DataFrame(show_list,columns=['question','source document','match document','match title']).to_excel('record.xlsx')

In [149]:
from sklearn.metrics import accuracy_score
db = pd.read_excel('./record.xlsx')
v1 = db.iloc[:,2]
v2 = db.iloc[:,3]
accuracy_score(v1,v2)

0.24

## 基于神经网络

### 准备数据集

In [228]:
feature =np.array([model.encode(iquestion) for iquestion in question_list])
label = np.array([question_list_copy.index(iquestion) // 100 + 1 for iquestion in question_list]).reshape(-1,1)

In [287]:
label_one_hot = []
class_type = 4
for i in range(label.shape[0]) :
    eye = np.zeros([4])
    eye[label[i]-1] = 1
    label_one_hot.append(eye)
label = np.array(label_one_hot)

In [644]:
np.concatenate([feature,label],axis=1).shape

(400, 772)

In [323]:
# 检查cuda是否可用
torch.cuda.is_available()
# 清空缓存
torch.cuda.empty_cache()

In [642]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

def score_func(batch_size,hidden_size,drop_prod,lr,epochs) :
    batch_size = int(batch_size)
    hidden_size = int(hidden_size)
    epochs = int(epochs)
    # 划分训练集
    trainset,testset = train_test_split(np.concatenate([feature,label],axis=1),test_size=0.3,shuffle=True)

    # 定义特征以及标签
    feature_train , label_train = trainset[:,0:trainset.shape[-1]-4] , trainset[:,trainset.shape[-1]-4:trainset.shape[-1]]
    feature_test , label_test = testset[:,0:testset.shape[-1]-4] , testset[:,testset.shape[-1]-4:testset.shape[-1]]

    # 定义数据集
    traindataset = torch.utils.data.TensorDataset(torch.Tensor(feature_train),torch.Tensor(label_train))
    testdataset = torch.utils.data.TensorDataset(torch.Tensor(feature_test),torch.Tensor(label_test))

    # 定义加载器
    # batch_size = 10
    traindataloader = torch.utils.data.DataLoader(traindataset,shuffle=True,batch_size=batch_size,drop_last=True)
    testdataloader = torch.utils.data.DataLoader(testdataset,shuffle=True,batch_size=batch_size,drop_last=True)

    # 定义神经网络结构
    class network(nn.Module) :
        def __init__(self,input_size,hidden_size,drop_prod) :
            super().__init__()
            self.model = nn.Sequential(
                nn.Linear(in_features = input_size,out_features = hidden_size,bias = True)
                nn.ReLU()
                nn.Dropout(p=drop_prod)
                nn.Linear(in_features = hidden_size,out_features = 2 * hidden_size,bias = True)
                nn.ReLU()
                nn.Dropout(p=drop_prod)
                nn.Linear(in_features = 2 * hidden_size,out_features = class_type,bias = True)
                nn.Dropout(p=drop_prod)
            )
        def forward(self,x) :
            return self.model(x)

    # 实例化网络
    input_size = feature.shape[-1]
    # hidden_size = 256
    # drop_prod = 0.8
    net = network(input_size=input_size,hidden_size=hidden_size,drop_prod=drop_prod)

    # 迁移模型
    # net.to("cuda:0")

    # 定义优化器
    # lr = 1e-3
    optimizer =torch.optim.Adam(net.parameters(),lr)

    # 定义损失函数
    loss = nn.CrossEntropyLoss()

    # 初始化参数
    for name,param in net.named_parameters() :
        if "bias" in name :
            nn.init.constant_(param,val=0)
        elif "weight" in name :
            nn.init.normal_(param,mean=0,std=0.01)

    # epochs = 3000
    loss_list = []
    for epoch in range(epochs) :
        ls = 0
        lt = 0
        a = 0
        for pi_X,pi_y in traindataloader :
            # 迁移数据
            # pi_X = pi_X.to("cuda:0")
            # pi_y = pi_y.to("cuda:0")
            l = loss(net(pi_X),pi_y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            ls += l
            with torch.no_grad():
                #开启评估模式
                net.eval()
                for pi_X_test,pi_y_test in testdataloader :
                    # pi_X_test = pi_X_test.to("cuda:0")
                    # pi_y_test = pi_y_test.to("cuda:0")
                    lt += loss(net(pi_X_test),pi_y_test)
                    a += ((net(pi_X_test).argmax(dim=1) == pi_y_test.argmax(dim=1))**2).sum()
            net.train()
    return -a , net

SyntaxError: invalid syntax (3344003829.py, line 31)

In [614]:
batch_size_bayes = (10,100)
hidden_size_bayes = (100,400)
drop_prod_bayes = (0.2,0.8)
lr_bayes = (1e-3,5e-2)
epochs_bayes = (1000,3000)

In [623]:
from bayes_opt import BayesianOptimization

score_func_score = lambda **kwargs : score_func(**kwargs)[0]

param_dict = {
    "batch_size":batch_size_bayes,
    "hidden_size":hidden_size_bayes,
    "drop_prod":drop_prod_bayes,
    "lr":lr_bayes,
    "epochs":epochs_bayes,
}
# opt = BayesianOptimization(score_func,param_dict)
opt = BayesianOptimization(score_func_score,param_dict)

In [ ]:
opt.maximize(n_iter=15, init_points=3)
print(opt.max)

In [639]:
# 使用最优化参数进行训练并且保存模型
_,mynet = score_func(**opt.max["params"])
mynet

network(
  (linear1): Linear(in_features=768, out_features=248, bias=True)
  (activefunc1): ReLU()
  (drop1): Dropout(p=0.779723130418032, inplace=False)
  (linear2): Linear(in_features=248, out_features=4, bias=True)
  (drop2): Dropout(p=0.779723130418032, inplace=False)
)

In [640]:
mynet.eval()
mynet(torch.Tensor(feature_test))

tensor([[-0.0155,  0.0232, -0.0615, -0.0226],
        [-0.0155,  0.0232, -0.0615, -0.0226],
        [-0.0155,  0.0232, -0.0615, -0.0226],
        [-0.0155,  0.0232, -0.0615, -0.0226],
        [-0.0155,  0.0232, -0.0615, -0.0226],
        [-0.0155,  0.0232, -0.0615, -0.0226],
        [-0.0155,  0.0232, -0.0615, -0.0226],
        [-0.0155,  0.0232, -0.0615, -0.0226],
        [-0.0155,  0.0232, -0.0615, -0.0226],
        [-0.0155,  0.0232, -0.0615, -0.0226],
        [-0.0155,  0.0232, -0.0615, -0.0226],
        [-0.0155,  0.0232, -0.0615, -0.0226],
        [-0.0155,  0.0232, -0.0615, -0.0226],
        [-0.0155,  0.0232, -0.0615, -0.0226],
        [-0.0155,  0.0232, -0.0615, -0.0226],
        [-0.0155,  0.0232, -0.0615, -0.0226],
        [-0.0155,  0.0232, -0.0615, -0.0226],
        [-0.0155,  0.0232, -0.0615, -0.0226],
        [-0.0155,  0.0232, -0.0615, -0.0226],
        [-0.0155,  0.0232, -0.0615, -0.0226],
        [-0.0155,  0.0232, -0.0615, -0.0226],
        [-0.0155,  0.0232, -0.0615

In [631]:
mynet(torch.Tensor(feature_test)).argmax(dim=1)

label_test.argmax(axis=1)

array([3, 3, 3, 0, 3, 3, 2, 2, 1, 3, 0, 3, 1, 0, 1, 1, 0, 1, 2, 2, 1, 1,
       0, 1, 1, 3, 2, 2, 0, 0, 2, 2, 1, 1, 1, 0, 0, 0, 3, 2, 0, 2, 1, 0,
       3, 2, 0, 1, 0, 2, 1, 3, 3, 1, 1, 1, 2, 3, 3, 3, 3, 2, 2, 1, 2, 0,
       2, 0, 2, 3, 2, 2, 1, 1, 1, 3, 0, 1, 1, 0, 3, 1, 2, 2, 1, 1, 2, 2,
       2, 0, 0, 3, 3, 1, 0, 0, 0, 3, 3, 2, 0, 0, 2, 3, 3, 1, 2, 2, 2, 0,
       0, 3, 1, 1, 1, 0, 3, 0, 2, 2], dtype=int64)

In [559]:
mynet.eval()
test_question = "Name some common Methanogenic bacteria."
test_question_encode = model.encode(test_question)
nn.Softmax()(mynet(torch.Tensor(np.ones([10,1]).dot(test_question_encode.reshape(1,-1))))).argmax(dim=1)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_37508\3827524707.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  nn.Softmax()(mynet(torch.Tensor(np.ones([10,1]).dot(test_question_encode.reshape(1,-1))))).argmax(dim=1)


tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [431]:
# feature_test.shape

# mynet(feature_test)

# for pi_X,pi_y in traindataloader :
#     print(pi_X.shape)
# mynet(pi_X)

mynet(torch.Tensor(feature_test))
torch.Tensor(feature_test).shape

torch.Size([120, 768])

In [ ]:
mynet(torch.Tensor(test_question_encode.reshape(-1,1)))

In [ ]:
# 首先根据question根据prompt进行分类

# 其次对参考文献做分类问题，算法为二次相似度检验算法


In [ ]:
def question_match(question) :
    # 对问题进行编码处理
    question_encode = model.encode(question)
    # 第一层检索
    # 根据每个prompt的title进行优先匹配
    prompt_title = np.array(["Anaerobic digestion (material addition effects)",
                        "Microbial effects of anaerobic digestion",
                        "Analysis of the nature of anaerobic digestion systems",
                        "Impact of anaerobic digestion on gas production",])
    title_encode = model.encode(prompt_title)
    one_vector = cosine_similarity(title_encode,question_encode.reshape(1,-1))
    # 第二层检索
    # 根据每个prompt的content进行二次匹配
        # 给出文件列表
    dir_list = [f'./t{i}.txt' for i in range(1,5)]
    content_list = []
        # 读取四种类型的prompt
    for dir in dir_list :
        with open(dir , 'r' , encoding='utf8') as f :
            content = f.read()
        content_list.append(content)
    content_list = model.encode(np.array(content_list))
    two_vector = cosine_similarity(content_list,question_encode.reshape(1,-1))

    # print(f"第一层认为第{one_vector.argmax()+1}个最匹配,第二层认为第{two_vector.argmax()+1}个最匹配,综合认为第{(0.8 * one_vector + 0.2 * two_vector).argmax() + 1}个最匹配")
    return (0.8 * one_vector + 0.2 * two_vector).argmax() + 1

In [381]:
# 文章关键词列表
pdf_keywords
# 文章摘要列表
pdf_subject

# 第一层检索
# 对pdf标题进行编码处理
pdf_keywords_encode = model.encode(pdf_keywords)

# 对prompt标题进行编码处理
prompt_title = np.array(["Anaerobic digestion (material addition effects)",
                    "Microbial effects of anaerobic digestion",
                    "Analysis of the nature of anaerobic digestion systems",
                    "Impact of anaerobic digestion on gas production",])
title_encode = model.encode(prompt_title)
one_vector_list = []

for i in range(pdf_keywords_encode.shape[0]) :
    one_vector = cosine_similarity(title_encode,pdf_keywords_encode[i:i+1,:])
    one_vector_list.append(one_vector.reshape(-1))
np.array(one_vector_list).shape     

In [ ]:
# 第二层检索
# 对pdf摘要进行编码处理
pdf_subject_encode = model.encode(pdf_subject)

# 对prompt内容进行编码处理
# 给出文件列表
dir_list = [f'./t{i}.txt' for i in range(1,5)]
content_list = []
# 读取四种类型的prompt
for dir in dir_list :
    with open(dir , 'r' , encoding='utf8') as f :
        content = f.read()
    content_list.append(content)
content_list = model.encode(np.array(content_list))

two_vector_list = []
for i in range(pdf_subject_encode.shape[0]) :
    one_vector = cosine_similarity(title_encode,pdf_subject_encode[i:i+1,:])
    one_vector_list.append(one_vector.reshape(-1))
np.array(two_vector_list).shape

# 利用llama-api推理

In [ ]:
import replicate
import pandas as pd
from langchain.prompts import PromptTemplate

In [ ]:
template="""
You are an expert in the field of anaerobic fermentation. You should answer in the following format based on a full understanding of the literature.
first, Briefly summarize the answers.
second, Answer the question in more detail and logically, explaining every detail of the question in more detail. The format is
    (1) Sub-explanation
    (2) Sub-explanation
etc...
Sub-explanation should not be limited to two, usually three or more;
finally, Summarize all the above conclusions and further explain the problem.

{context}

{chat_history}
The question you are about to answer is:
{question}
answer:

"""

In [ ]:
prompt = PromptTemplate(input_variables=["chat_history","context","question"], template=template)
prompt.format(question = question , context = document , chat_history = [])

pc = template.format(question = question , context = document , chat_history = [])

In [ ]:
input_kwarg = {
    "prompt": pc,
    "top_p":1,
    # "max_length":8000,
    "temperature":0.75,
    "repetition_penalty":1,
    "max_new_tokens":1000,
}

o1 = replicate.run(
        "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
        # "meta/llama-2-13b-chat:f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d",
        # "meta/llama-2-7b:527827021d8756c7ab79fde0abbfaac885c37a3ed5fe23c7465093f0878d55ef",
        # input={"prompt": "What effect does the addition of biochar have on the biogas production of anaerobic digestion of food waste? What should its optimal addition amount be?"}
        input=input_kwarg
    )

In [ ]:
# "".join(list(o1))
import time
while True:
    time.sleep(2)
    print(o1.__next__())